In [1]:
!pip install -U optuna-integration[tfkeras]
!pip install -U pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\79112\AppData\Local\Temp\pip-install-lpx99y62\pytorch_af256cc7e154476c82043e098369c0e2\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [1]:
import pandas as pd
import numpy as np
import warnings

from feature_engineering import AAVE, ARB, BNB, BTC, ETH, JASMY, LDO, LINK, OP, PENDLE
from data_preprocessing import prepare_data, prepare_for_train
from vizualize import show_example


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Lambda, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

In [81]:
warnings.simplefilter('ignore', Warning)

In [2]:
df = prepare_data(AAVE, multiplier=2).copy()
df

,timestamp,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,next_high,next_low,next_close,dynamic_range,target
199,2023-01-02 22:45:00,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,53.50,53.20,53.30,0.004214,1
200,2023-01-02 23:00:00,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,53.40,53.30,53.30,0.004396,0
201,2023-01-02 23:15:00,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,53.40,53.20,53.40,0.004310,0
202,2023-01-02 23:30:00,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,53.40,52.90,53.00,0.003978,-1
203,2023-01-02 23:45:00,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,53.10,52.90,53.00,0.003977,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34945,2023-12-30 22:30:00,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,112.48,111.33,112.09,0.006459,-1
34946,2023-12-30 22:45:00,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,112.95,112.02,112.45,0.006363,1
34947,2023-12-30 23:00:00,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,112.51,112.03,112.15,0.007418,0
34948,2023-12-30 23:15:00,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,112.33,111.49,111.50,0.007028,0


In [3]:
show_example(df)

In [4]:
prepare_for_train(df).reset_index(drop=True)

,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,target
0,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,1
1,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,0
2,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,0
3,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,-1
4,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34746,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,-1
34747,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,1
34748,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,0
34749,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,0


In [5]:
df.drop('timestamp', axis=1, inplace=True)

In [6]:
df['target'] = df['target'].replace({-1: 1, 1: 2}) 

In [7]:
df = prepare_for_train(df)
df

,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,target
199,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,2
200,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,0
201,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,0
202,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,1
203,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34945,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,1
34946,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,2
34947,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,0
34948,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,0


In [54]:
class_weights = df.target.value_counts()
class_weights

(0.1273344651952462, 0.7554027222238209, 0.11726281258093292)

In [62]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class FinancialDataset(Dataset):
    def __init__(self, data, sequence_length, target_col):
        """
        data: pandas DataFrame с временными рядами и признаками
        sequence_length: длина временной последовательности
        target_col: название столбца, который является целевой переменной
        """
        self.data = data
        self.sequence_length = sequence_length
        self.target_col = target_col

        # Преобразуем данные в тензоры
        self.features = torch.tensor(data.drop(columns=[target_col]).values, dtype=torch.float32)
        self.targets = torch.tensor(data[target_col].values, dtype=torch.float32)

    def __len__(self):
        # Количество доступных последовательностей
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        # Последовательность признаков
        x = self.features[idx:idx + self.sequence_length]
        # Целевая переменная для предсказания
        y = self.targets[idx + self.sequence_length]
        # Добавляем размерность для y
        y = torch.tensor([y], dtype=torch.float32)  # Теперь y имеет форму [1]
        return x, y

In [63]:
targets = df.target.values
features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 

# Разделение на тренировочные и валидационные данные
x_train, x_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, shuffle=False)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Создание Dataset
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score # <---  Добавлен импорт
import numpy as np

# === Класс для взвешенной функции потерь ===
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()  # Преобразуем в long
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

# === Класс для LSTM ===
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_last_out = lstm_out[:, -1, :]  # Последний временной шаг
        output = self.fc(lstm_last_out)
        return output

# === Функция обучения ===
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).long()  # Приведение к long
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        all_val_targets = [] # Добавлено для сбора целей валидации
        all_val_outputs = [] # Добавлено для сбора предсказаний валидации
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device).long()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                all_val_targets.extend(targets.cpu().numpy()) # Сбор целей
                all_val_outputs.extend(torch.argmax(outputs, dim=1).cpu().numpy()) # Сбор предсказаний

        avg_val_loss = val_loss / len(val_loader)

        # Расчет метрик на валидационном наборе
        val_accuracy = accuracy_score(all_val_targets, all_val_outputs)
        val_f1_macro = f1_score(all_val_targets, all_val_outputs, average='macro', zero_division=0) # Важно zero_division=0

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Macro: {val_f1_macro:.4f}") # Вывод метрик

    return model

# === Подготовка данных ===
# Генерация фиктивных данных (замените на ваш DataFrame df)
num_samples = 1000
num_features = 10
num_classes = 3

np.random.seed(42)
features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 
targets = df.target.values

# Разделение на train и val
x_train, x_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, shuffle=False)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

# Создание Dataset и DataLoader
train_dataset = TensorDataset(x_train.unsqueeze(1), y_train)  # Добавляем временную размерность
val_dataset = TensorDataset(x_val.unsqueeze(1), y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# === Инициализация модели ===
input_size = x_train.shape[1]  # Количество признаков
hidden_size = 64
num_layers = 2
num_classes = 3
class_weights = np.bincount(y_train.numpy()) / len(y_train)

model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = WeightedCrossEntropyLoss(class_weights)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-3,
    amsgrad=True,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Обучение модели ===
num_epochs = 20
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


Epoch 1/20, Train Loss: 0.2595, Val Loss: 0.2751, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 2/20, Train Loss: 0.2490, Val Loss: 0.2746, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 3/20, Train Loss: 0.2470, Val Loss: 0.2742, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 4/20, Train Loss: 0.2455, Val Loss: 0.2725, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 5/20, Train Loss: 0.2446, Val Loss: 0.2712, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 6/20, Train Loss: 0.2439, Val Loss: 0.2697, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 7/20, Train Loss: 0.2434, Val Loss: 0.2685, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 8/20, Train Loss: 0.2430, Val Loss: 0.2683, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 9/20, Train Loss: 0.2428, Val Loss: 0.2675, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 10/20, Train Loss: 0.2424, Val Loss: 0.2672, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 11/20, Train Loss: 0.2421, Val Loss: 0.2664, Val Accuracy: 0.74

---

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size // 2, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Get last time step output
        last_hidden = lstm_out[:, -1, :]
        
        # Apply batch normalization
        normalized = self.batch_norm(last_hidden)
        
        # First dense layer
        out = self.fc1(normalized)
        out = self.relu(out)
        out = self.dropout(out)
        
        # Output layer
        out = self.fc2(out)
        return out

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=5):
    model = model.to(device)
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            
            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(targets.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_metrics = calculate_metrics(np.array(train_targets), np.array(train_predictions))

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.squeeze())
                
                val_loss += loss.item()
                val_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(targets.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_metrics = calculate_metrics(np.array(val_targets), np.array(val_predictions))

        # Early stopping check
        if val_metrics['f1_macro'] > best_val_f1:
            best_val_f1 = val_metrics['f1_macro']
            patience_counter = 0
        else:
            patience_counter += 1

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {avg_train_loss:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1_macro']:.4f}")
        print(f"Val - Loss: {avg_val_loss:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_macro']:.4f}")
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    return model

def calculate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro', labels=[0, 1, 2], zero_division=0),
        'precision': precision_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0),
        'recall': recall_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0)
    }


# Гиперпараметры
input_size = 10  # Размерность входных данных
hidden_size = 128
num_layers = 2
num_classes = 3
learning_rate = 0.0001
num_epochs = 100
batch_size = 32

# Веса классов для несбалансированных данных
class_weights = [1.0, 1.0, 1.0]  # Настройте в соответствии с распределением классов

# Инициализация модели и оптимизатора
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = WeightedCrossEntropyLoss(class_weights)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


Epoch 1/100
Train - Loss: 0.7404, Accuracy: 0.7556, F1: 0.2902
Val - Loss: 0.8516, Accuracy: 0.7487, F1: 0.2854
Epoch 2/100
Train - Loss: 0.7207, Accuracy: 0.7572, F1: 0.2889
Val - Loss: 0.8796, Accuracy: 0.7006, F1: 0.3359
Epoch 3/100
Train - Loss: 0.7106, Accuracy: 0.7568, F1: 0.2918
Val - Loss: 0.9245, Accuracy: 0.7054, F1: 0.3350
Epoch 4/100
Train - Loss: 0.7027, Accuracy: 0.7566, F1: 0.2965
Val - Loss: 0.9762, Accuracy: 0.7282, F1: 0.3379
Epoch 5/100
Train - Loss: 0.6983, Accuracy: 0.7572, F1: 0.3018
Val - Loss: 1.0367, Accuracy: 0.7261, F1: 0.3358
Epoch 6/100
Train - Loss: 0.6936, Accuracy: 0.7572, F1: 0.3057
Val - Loss: 0.9514, Accuracy: 0.7395, F1: 0.3303
Epoch 7/100
Train - Loss: 0.6873, Accuracy: 0.7584, F1: 0.3212
Val - Loss: 1.1080, Accuracy: 0.7218, F1: 0.3429
Epoch 8/100
Train - Loss: 0.6860, Accuracy: 0.7581, F1: 0.3266
Val - Loss: 1.1035, Accuracy: 0.7300, F1: 0.3405
Epoch 9/100
Train - Loss: 0.6809, Accuracy: 0.7592, F1: 0.3383
Val - Loss: 1.1337, Accuracy: 0.7196, F1:

KeyboardInterrupt: 

---

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Инициализация скрытого и клеточного состояний
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через LSTM
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Получение последнего временного шага
        last_hidden = lstm_out[:, -1, :]
        
        # Пропуск через Dropout
        out = self.dropout(last_hidden)
        
        # Выходной слой
        out = self.fc(out)
        return out

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=10):
    model = model.to(device)
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Фаза обучения
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            
            loss.backward()
            # Ограничение градиентов для предотвращения взрыва градиентов
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(targets.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_metrics = calculate_metrics(np.array(train_targets), np.array(train_predictions))

        # Фаза валидации
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.squeeze())
                
                val_loss += loss.item()
                val_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(targets.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_metrics = calculate_metrics(np.array(val_targets), np.array(val_predictions))

        # Раннее прекращение обучения
        if val_metrics['f1_macro'] > best_val_f1:
            best_val_f1 = val_metrics['f1_macro']
            patience_counter = 0
            # Сохранение лучшей модели
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {avg_train_loss:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1_macro']:.4f}")
        print(f"Val   - Loss: {avg_val_loss:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_macro']:.4f}")
        
        if patience_counter >= patience:
            print("Ранняя остановка активирована")
            break

    # Загрузка лучшей модели
    model.load_state_dict(torch.load('best_model.pth'))
    return model

def calculate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro', labels=[0, 1, 2], zero_division=0),
        'precision': precision_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0),
        'recall': recall_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0)
    }

def prepare_data(file_paths):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler, LabelEncoder

    features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 
    targets = df.target.values
    
    # Нормализация
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    
    all_data = [features, targets]
    
    # Объединение всех данных
    X = all_data[0]
    y = all_data[1]
    
    return X, y

def create_data_loaders(X, y, sequence_length=10, batch_size=32, split_ratio=0.8):
    # Создание последовательностей для LSTM
    sequences = []
    targets = []
    for i in range(len(X) - sequence_length):
        sequences.append(X[i:i+sequence_length])
        targets.append(y[i+sequence_length])
    
    sequences = np.array(sequences)
    targets = np.array(targets)
    
    # Разделение на обучающую и валидационную выборки
    split = int(len(sequences) * split_ratio)
    X_train, X_val = sequences[:split], sequences[split:]
    y_train, y_val = targets[:split], targets[split:]
    
    # Создание TensorDataset
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    
    # Создание DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# Основной блок кода
if __name__ == "__main__":
    # Пути к файлам с данными
    file_paths = sv_files = [f for f in os.listdir(
        "./data/") if os.path.isfile(os.path.join("./data/", f))]
    
    # Подготовка данных
    X, y = prepare_data(file_paths)
    
    # Создание DataLoader
    train_loader, val_loader = create_data_loaders(X, y, sequence_length=10, batch_size=32, split_ratio=0.8)
    
    # Гиперпараметры
    input_size = X.shape[1]  # Количество признаков
    hidden_size = 64
    num_layers = 2
    num_classes = 3
    learning_rate = 0.0005
    num_epochs = 100
    batch_size = 32
    
    # Рассчёт весов классов для обработки дисбаланса
    class_counts = np.bincount(y)
    class_weights = 1. / (class_counts + 1e-6)  # Добавление малой величины для избежания деления на ноль
    class_weights = class_weights / class_weights.sum() * len(class_weights)  # Нормализация
    
    # Инициализация модели и оптимизатора
    model = LSTMModel(input_size, hidden_size, num_layers, num_classes, dropout=0.3)
    criterion = WeightedCrossEntropyLoss(class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Обучение модели
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=10)
    
    # Сохранение финальной модели
    torch.save(model.state_dict(), 'final_model.pth')


Epoch 1/100
Train - Loss: 1.0848, Accuracy: 0.5025, F1: 0.3472
Val   - Loss: 1.0800, Accuracy: 0.5053, F1: 0.3497
Epoch 2/100
Train - Loss: 1.0764, Accuracy: 0.5263, F1: 0.3655
Val   - Loss: 1.0782, Accuracy: 0.5214, F1: 0.3145
Epoch 3/100
Train - Loss: 1.0722, Accuracy: 0.5464, F1: 0.3698
Val   - Loss: 1.0784, Accuracy: 0.5091, F1: 0.3357
Epoch 4/100
Train - Loss: 1.0704, Accuracy: 0.5369, F1: 0.3731
Val   - Loss: 1.0742, Accuracy: 0.5018, F1: 0.3351
Epoch 5/100
Train - Loss: 1.0697, Accuracy: 0.5379, F1: 0.3760
Val   - Loss: 1.0769, Accuracy: 0.4465, F1: 0.3201
Epoch 6/100
Train - Loss: 1.0670, Accuracy: 0.5405, F1: 0.3798
Val   - Loss: 1.0781, Accuracy: 0.5424, F1: 0.3383
Epoch 7/100
Train - Loss: 1.0663, Accuracy: 0.5489, F1: 0.3858
Val   - Loss: 1.0883, Accuracy: 0.4064, F1: 0.2876
Epoch 8/100
Train - Loss: 1.0656, Accuracy: 0.5456, F1: 0.3837
Val   - Loss: 1.0783, Accuracy: 0.4929, F1: 0.3366
Epoch 9/100
Train - Loss: 1.0647, Accuracy: 0.5463, F1: 0.3798
Val   - Loss: 1.0755, Acc

C:\Users\79112\AppData\Local\Temp\ipykernel_20068\3906777930.py:124: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



------------------------

In [ ]:
def scale_data(dummy_data):
    """Масштабирование данных."""
    scaler = RobustScaler()
    return scaler.fit_transform(dummy_data)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Динамический график изменения learning rate с warm-up."""
    def __init__(self, initial_lr=1e-4, warmup_steps=1000):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return tf.minimum(self.initial_lr * tf.sqrt(step / self.warmup_steps), self.initial_lr)

def weighted_asymmetric_loss(y_true, y_pred):
    """Асимметричная взвешенная cross-entropy loss."""
    true_idx = tf.argmax(y_true, axis=1)
    weights = tf.where(true_idx == 2, 1.5, tf.where(true_idx == 1, 0.7, 1.0))
    ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(ce * weights)

def mare_loss(y_true, y_pred):
    """Mean Absolute Relative Error (MARE)."""
    return tf.reduce_mean(tf.abs((y_true - y_pred) / (y_true + 1e-8)))

class CustomLoss(tf.keras.losses.Loss):
    """Комбинированная функция потерь."""
    def __init__(self, alpha=0.3, beta=0.2, name="custom_loss"):
        super().__init__(name=name)
        self.alpha = alpha
        self.beta = beta

    def call(self, y_true, y_pred):
        direction_true, high_true, low_true = y_true
        direction_pred, high_pred, low_pred = y_pred

        ce_loss = weighted_asymmetric_loss(direction_true, direction_pred)
        hl_loss = 0.5 * (mare_loss(high_true, high_pred) + mare_loss(low_true, low_pred))
        return ce_loss + self.alpha * hl_loss

def prepare_data(data, window_size=30):
    X, y_dir, y_high, y_low = [], [], [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size : i])
        y_dir.append(data[i, 0:3])
        y_high.append(data[i, 3])  # High
        y_low.append(data[i, 4])   # Low

    # Возвращаем данные ПОСЛЕ завершения цикла
    X = np.array(X)
    y_dir = np.array(y_dir)
    y_high = np.array(y_high).reshape(-1, 1)
    y_low = np.array(y_low).reshape(-1, 1)
    
    return X, [y_dir, y_high, y_low]

def create_lstm_model(input_shape, n_units=64, freeze_encoder=False):
    """Создание мультивыходной LSTM-модели."""
    inputs = Input(shape=input_shape, name="input_layer")
    x = LSTM(n_units, return_sequences=True, name="lstm")(inputs)

    # Направление
    direction_branch = LSTM(n_units // 2, return_sequences=False, name="lstm2")(x)
    direction_out = Dense(3, activation="softmax", name="direction")(direction_branch)

    # Прогноз High/Low
    atr = Lambda(lambda x: x[:, :, -1:], name="atr_slice")(inputs)  # Последний столбец — ATR
    atr = Reshape((input_shape[0], 1), name="atr_reshape")(atr)     # Приводим к совместимой форме
    hl_concat = concatenate([x, atr], axis=-1, name="concat_atr")

    hl_hidden = Dense(32, activation="selu", name="hl_dense")(hl_concat)
    high_out = Dense(1, activation="linear", name="high")(hl_hidden)
    low_out = Dense(1, activation="linear", name="low")(hl_hidden)

    model = Model(inputs=inputs, outputs=[direction_out, high_out, low_out])
    if freeze_encoder:
        for layer in model.layers[:-6]:
            layer.trainable = False

    return model

# -------------------------------------------------------------------------
# 2. Подготовка данных
# -------------------------------------------------------------------------

# Сгенерируем данные для примера
np.random.seed(42)  # Для воспроизводимости
dummy_data = np.random.rand(1000, 16)  # Пример ваших данных (1000 наблюдений, 16 признаков)

# Масштабирование
scaled_data = scale_data(dummy_data)

# Создание временных окон
window_size = 30
X_seq, y_seq = prepare_data(scaled_data, window_size=window_size)

# Разделение данных
total_samples = X_seq.shape[0]
train_end = int(0.7 * total_samples)
val_end = int(0.85 * total_samples)

X_train = X_seq[:train_end]
X_val = X_seq[train_end:val_end]
X_test = X_seq[val_end:]

y_dir_train, y_high_train, y_low_train = y_seq[0][:train_end], y_seq[1][:train_end], y_seq[2][:train_end]
y_dir_val,   y_high_val,   y_low_val   = y_seq[0][train_end:val_end], y_seq[1][train_end:val_end], y_seq[2][train_end:]
y_dir_test,  y_high_test,  y_low_test  = y_seq[0][val_end:], y_seq[1][val_end:], y_seq[2][val_end:]

# -------------------------------------------------------------------------
# 3. Создание и обучение модели
# -------------------------------------------------------------------------

# Создание модели
model = create_lstm_model(input_shape=(window_size, 16), n_units=64)

# Компиляция модели
lr_schedule = CustomSchedule()
optimizer = Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss={
        "direction": weighted_asymmetric_loss,
        "high": "mae",
        "low": "mae"
    },
    loss_weights=[0.7, 0.15, 0.15],
    metrics={
        "direction": "accuracy",
        "high": "mae",
        "low": "mae"
    }
)

# Обучение модели
train_targets = {"direction": y_dir_train, "high": y_high_train, "low": y_low_train}
val_targets = {"direction": y_dir_val, "high": y_high_val, "low": y_low_val}

callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1),
    EarlyStopping(monitor="val_direction_accuracy", patience=10, verbose=1, restore_best_weights=True)
]

history_stage1 = model.fit(
    X_train,
    train_targets,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, val_targets),
    callbacks=callbacks,
    verbose=1
)

# -------------------------------------------------------------------------
# 4. Оценка модели
# -------------------------------------------------------------------------

print("\nModel Evaluation:")
test_results = model.evaluate(X_test, {"direction": y_dir_test, "high": y_high_test, "low": y_low_test})
print("Test Results:", test_results)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - direction_accuracy: 0.3459 - direction_loss: 0.0056 - high_loss: 0.5104 - high_mae: 0.5105 - loss: 0.1604 - low_loss: 0.5323 - low_mae: 0.5325  

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 145
'y' sizes: 145, 145, 291
